# Logistic Regression, SVMs, and Gradient Optimization

*Please address questions to Professor Eric Larson, eclarson@smu.edu*

In this notebook we will explore methods of using logistic regression in `scikit-learn` and we will also investigate methods for gradient descent. Finally we will look at using support vector machines and investigate parameters of kernel functions. A basic understanding of `scikit-learn` is required to complete this notebook, but we start very basic. Note also that there are more efficient methods of separating testing and training data, but we will leave that for a later lecture. 

First let's load a dataset and prepare it for analysis. We will use pandas to load in data, and then prepare it for classification. We will be using the titanic dataset (a very modest sized data set of about 1000 instances) before loading a larger, more complicated dataset for gradient descent methods.

______
The imputation methods used here are discussed in a previous notebook. 

In [1]:
from __future__ import print_function

import pandas as pd
import numpy as np

df = pd.read_csv('data/owid-covid-data_modified.csv') # read in the csv file


________
Now let's look a little further at each of the categorical objects. Note that age range has already been saved as an ordinal. We need to look at `Sex` and `Embarked` objects. 

Because the `Sex`  attribute binary, there is no need to encode it using OneHotEncoding. We can just convert it to an integer. However, we should transform the `Embarked` attribute to take on three different values--one for each possible variable outcome.

___
# Training and Testing Split
For training and testing purposes, let's gather the data we have and grab 80% of the instances for training and the remaining 20% for testing. Moreover, let's repeat this process of separating the testing and training data three times. We will use the hold out cross validation method built into `scikit-learn`.

In [2]:
# Ideas from https://scikit-learn.org/stable/modules/cross_validation.html#timeseries-cv

# Sorting data frame by date column
df['date'] = pd.to_datetime(df['date']) # Converting data columnn to datetime

df = df.sort_values(by='date', ascending=True)

df

,Unnamed: 0,continent,location,date,new_cases,new_cases_smoothed,new_deaths,new_deaths_smoothed,reproduction_rate,new_vaccinations_smoothed,...,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,stringency_range,new_cases_range,new_deaths_range
0,493,Asia,Afghanistan,2021-07-01,1557.0,1526.286,91.0,72.857,0.980000,14115.000000,...,1803.987,597.029,9.59,37.746000,0.500000,64.83,0.511000,40-50,high,high
3515,16243,North America,Bermuda,2021-07-01,0.0,1.429,0.0,0.000,0.323548,430.000000,...,50669.315,139.547,13.00,11.404742,0.506452,82.59,0.545586,30-40,low,low
13690,57963,North America,Guatemala,2021-07-01,2855.0,1390.000,67.0,55.429,1.160000,8217.000000,...,7423.808,155.898,10.18,76.665000,0.600000,74.30,0.663000,50-60,high,high
21275,91453,North America,Mexico,2021-07-01,6081.0,4609.000,201.0,168.571,1.230000,522893.000000,...,17336.469,152.783,13.06,87.847000,1.380000,75.05,0.779000,40-50,high,high
31265,136536,Asia,Tajikistan,2021-07-01,24.0,21.714,0.0,0.000,1.530000,14170.000000,...,2896.913,427.698,7.11,72.704000,4.800000,71.10,0.668000,30-40,low,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14429,60502,South America,Guyana,2022-01-01,206.0,99.000,0.0,1.286,1.960000,839.000000,...,7435.047,373.159,11.62,77.159000,1.600000,69.91,0.682000,50-60,medium,low
3144,14433,Europe,Belgium,2022-01-01,0.0,9604.571,0.0,26.000,1.340000,45720.000000,...,42658.576,114.898,4.29,90.078650,5.640000,81.63,0.931000,40-50,low,low
31819,138128,Asia,Thailand,2022-01-01,3011.0,2715.429,10.0,21.429,1.180000,64643.666667,...,16277.671,109.861,7.04,90.670000,2.100000,77.15,0.777000,40-50,high,high
26269,113951,Asia,Qatar,2022-01-01,833.0,509.286,0.0,0.571,2.230000,7892.000000,...,116935.600,176.690,16.52,34.203248,1.200000,80.23,0.848000,30-40,high,low


In [3]:
print(df.columns.tolist())

['Unnamed: 0', 'continent', 'location', 'date', 'new_cases', 'new_cases_smoothed', 'new_deaths', 'new_deaths_smoothed', 'reproduction_rate', 'new_vaccinations_smoothed', 'new_people_vaccinated_smoothed', 'stringency_index', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'cardiovasc_death_rate', 'diabetes_prevalence', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'stringency_range', 'new_cases_range', 'new_deaths_range']


In [4]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

#number = LableEncoder()
#train[''] = number.fit_transform(train[''].astype('str'))
#test[''] = number.fit_transform(test[''].astype('str'))
#train.head(5)

#columns = “new_cases new_deaths new_cases_smoothed new_deaths_smoothed reproduction_rate cardiovasc_death_rate handwashing_facilities stringency_range 	human_development_index”.split() # Declare the columns names
#df = pd.DataFrame(df.data, columns=columns) # load the dataset as a pandas data frame
#dataset = df()
#df = df.drop(columns=['continent', 'location','stringency_range','new_cases_range','new_deaths_range','new_vaccinations_smoothed','new_people_vaccinated_smoothed','population','population_density','hospital_beds_per_thousand'])
#df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#df['date'] = pd.to_numeric(pd.to_datetime(df['date']))

#df2=df.drop(['date','continent','location','new_deaths_range','stringency_range','new_cases_range'], axis=1)
#df2

df2 = df.drop(["continent","date","location","stringency_range","new_cases_range"], axis=1)
df2

#del df['date']
#del df['location']
#del df['new_deaths_range']
#del df['stringency_range']
#del df['new_cases_range']
#del df['continent']

#y = df.new_cases # define the target variable (dependent variable) as y
#del df['new_cases'] # get rid of the class label
#X = df.values # use everything else to predict!

# we want to predict the X and y data as follows:
#if 'new_deaths_range' in float_df:
#    y = float_df['new_deaths_range'].values # get the labels we want
#   del float_df['new_deaths_range'] # get rid of the class label
#   X = float_df.values # use everything else to predict!
#      ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn

# create training and testing vars
#X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
#print(X_train.shape, y_train.shape)

# we want to predict the X and y data as follows:
# i have commented out below code
# we want to predict the X and y data as follows:
# we want to predict the X and y data as follows:
if 'new_deaths_range' in df2:
    y = df2['new_deaths_range'].values # get the labels we want
    del df2['new_deaths_range'] # get rid of the class label
    X = df2.values # use everything else to predict!
       ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
        
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                       test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


___
# Logistic Regression
Now let's use Logistic Regression from `scikit-learn`. The documentation can be found here:

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [5]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None, solver='liblinear') # get object
#lr_clf = LogisticRegression(random_state=0).fit(X, y)

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

# Converting string to float
#df['continent'] = pd.to_numeric(df['continent'])
#df['continent'] = df['continent'].astype(int)
#df['continent'] = pd.to_numeric(df['continent'])

#df['continent'] = df['continent'].astype(float)
#df['location'] = df['location'].astype(float)

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y):   
# I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]

 # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
 # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.6437866437866437
confusion matrix
 [[1815  471   37]
 [  99 2669   11]
 [ 233 1640   18]]
====Iteration 1  ====
accuracy 0.6424996424996425
confusion matrix
 [[1790  546    7]
 [  91 2699    1]
 [ 215 1640    4]]
====Iteration 2  ====
accuracy 0.642070642070642
confusion matrix
 [[1791  525   25]
 [ 115 2690    1]
 [ 227 1610    9]]


In [6]:
# this does the exact same thing as the above block of code, but with shorter syntax

for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    lr_clf.fit(X[train_indices],y[train_indices])  # train object
    y_hat = lr_clf.predict(X[test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num," ====")
    print("accuracy", mt.accuracy_score(y[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(y[test_indices],y_hat))

====Iteration 0  ====
accuracy 0.6330616330616331
confusion matrix
 [[1702  618    0]
 [ 107 2725    0]
 [ 172 1669    0]]
====Iteration 1  ====
accuracy 0.636922636922637
confusion matrix
 [[1715  545    0]
 [  98 2739    0]
 [ 220 1676    0]]
====Iteration 2  ====
accuracy 0.6593736593736593
confusion matrix
 [[1845  423  106]
 [ 108 2680   13]
 [ 241 1491   86]]


In [7]:
# and here is an even shorter way of getting the accuracies for each training and test set
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(lr_clf, X, y=y, cv=cv_object) # this also can help with parallelism
print(accuracies)

[0.65408265 0.64106964 0.65594166]


In [8]:
# here we can change some of the parameters interactively
from ipywidgets import widgets as wd

def lr_explor(cost):
    lr_clf = LogisticRegression(penalty='l2', C=cost, class_weight=None,solver='liblinear') # get object
    accuracies = cross_val_score(lr_clf,X,y=y,cv=cv_object) # this also can help with parallelism
    print(accuracies)

wd.interact(lr_explor,cost=(0.001,5.0,0.05),__manual=True)

interactive(children=(FloatSlider(value=2.451, description='cost', max=5.0, min=0.001, step=0.05), Output()), …

<function __main__.lr_explor(cost)>

## Interpretting weights
Okay, so now lets take the last trained model for logistic regression and try to interpret the weights for the model. Is there something about the weights that makes this model more interpretable?

In [9]:
# interpret the weights

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = df.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])
    
# does this look correct? 

Unnamed: 0 has weight of -1.078309402315385e-05
continent has weight of 0.0005209026078943367
location has weight of 0.0004188316689318398
date has weight of 1.61547267766295e-05
new_cases has weight of 1.2918000843183024e-05
new_cases_smoothed has weight of -1.5436618941233674e-07
new_deaths has weight of -2.660914366613069e-07
new_deaths_smoothed has weight of 1.1842238335490325e-06
reproduction_rate has weight of -2.7826255977150727e-06
new_vaccinations_smoothed has weight of -2.0622984603898725e-09
new_people_vaccinated_smoothed has weight of -7.785761439742199e-05
stringency_index has weight of -1.4429996405887245e-06
population has weight of 3.734695632118015e-07
population_density has weight of 2.5263797941706303e-07
median_age has weight of -4.44866271739712e-05
aged_65_older has weight of -3.7092644541476205e-05
aged_70_older has weight of -8.271329651446903e-07
gdp_per_capita has weight of -6.444466718350313e-07
cardiovasc_death_rate has weight of -4.352029767374501e-08
diabe

These weight interpretations **are not neccessarily interpretable** because of the values we had. Very large attribute values could just as easily be assigned a higher weight. Instead, let's normalize the feature values so that all the attributes are on the same dynamic range. Once we normalize the attributes, the weights should have magnitudes that reflect their poredictive power in the logistic regression model.
___

In [10]:
from sklearn.preprocessing import StandardScaler

# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)

# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=0.05, solver='liblinear') # get object, the 'C' value is less (can you guess why??)
lr_clf.fit(X_train_scaled,y_train)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf )

# sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,df.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out

accuracy: 0.7225797225797226
[[2003  247   91]
 [  46 2471  289]
 [  41 1226  579]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# now let's make a pandas Series with the names and values, and plot them
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


weights = pd.Series(lr_clf.coef_[0],index=df_imputed.columns)
weights.plot(kind='bar')
plt.show()

**For more improvement and guarding against overfitting:** At this point it would make sense to remove variables that are highly related to one another or ones that are irrelevant and keep going with the weights analysis. What variables would you remove?
___

In [ ]:
from sklearn.preprocessing import StandardScaler
# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)

from sklearn.pipeline import Pipeline
# you can apply the StandardScaler function inside of the cross-validation loop 
#  but this requires the use of PipeLines in scikit. 
#  A pipeline can apply feature pre-processing and data fitting in one compact notation
#  Here is an example!

std_scl = StandardScaler()
lr_clf = LogisticRegression(penalty='l2', C=0.05, solver='liblinear') 

# create the pipline
piped_object = Pipeline([('scale', std_scl),  # do this
                         ('logit_model', lr_clf)]) # and then do this

weights = []
# run the pipline cross validated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    piped_object.fit(X[train_indices],y[train_indices])  # train object
    # it is a little odd getting trained objects from a  pipeline:
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    

weights = np.array(weights)

In [ ]:
import plotly
plotly.offline.init_notebook_mode() # run at the start of every notebook

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': df_imputed.columns,
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)

**For more improvement and guarding against overfitting:** At this point it would make sense to remove variables that are highly related to one another or ones that are irrelevant and keep going with the weights analysis. What variables would you remove?

In [ ]:
Xnew = df_imputed[['Age','Pclass','IsMale']].values

weights = []
# run the pipline corssvalidated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(Xnew,y)):
    piped_object.fit(Xnew[train_indices],y[train_indices])  # train object
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    
weights = np.array(weights)

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': ['Age','Pclass','IsMale'],
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)

# Support Vector Machines
We can use the previous training and testing attributes (scaled) to investigate the weights and support vectors in the attributes. SVMs were first hypothesized by Vladmir Vapnik ~50 years ago, but did not gain popularity until the turn of the millenium. 

In [ ]:
from IPython.display import Image
# Here he is, in all his glory:
Image(url='http://engineering.columbia.edu/files/engineering/vapnik.jpg')
# Image(url='http://yann.lecun.com/ex/images/allyourbayes.jpg')

In [ ]:
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) 

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels
from sklearn.svm import SVC

# train the model just as before
svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto') # get object
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)

In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

In [ ]:
# if using linear kernel, these make sense to look at (not otherwise, why?)
print(svm_clf.coef_)
weights = pd.Series(svm_clf.coef_[0],index=df_imputed.columns)
weights.plot(kind='bar')

In [ ]:
# Now let's do some different analysis with the SVM and look at the instances that were chosen as support vectors

# now lets look at the support for the vectors and see if we they are indicative of anything
# grabe the rows that were selected as support vectors (these are usually instances that are hard to classify)

# make a dataframe of the training data
df_tested_on = df_imputed.iloc[train_indices].copy() # saved from above, the indices chosen for training
# now get the support vectors from the trained model
df_support = df_tested_on.iloc[svm_clf.support_,:].copy()

df_support['Survived'] = y[svm_clf.support_] # add back in the 'Survived' Column to the pandas dataframe
df_imputed['Survived'] = y # also add it back in for the original data
df_support.info()

In [ ]:
# now lets see the statistics of these attributes
from pandas.plotting import boxplot

# group the original data and the support vectors
df_grouped_support = df_support.groupby(['Survived'])
df_grouped = df_imputed.groupby(['Survived'])

# plot KDE of Different variables
vars_to_plot = ['Age','Pclass','IsMale','FamilySize']

for v in vars_to_plot:
    plt.figure(figsize=(10,4))
    # plot support vector stats
    plt.subplot(1,2,1)
    ax = df_grouped_support[v].plot.kde() 
    plt.legend(['Perished','Survived'])
    plt.title(v+' (Instances chosen as Support Vectors)')
    
    # plot original distributions
    plt.subplot(1,2,2)
    ax = df_grouped[v].plot.kde() 
    plt.legend(['Perished','Survived'])
    plt.title(v+' (Original)')



So the analysis here is basically telling us what the original statistics of the data looked like, and also what the statistics of the support vectors looked like. We can see that the separation in distributions is not as great as the separation for the original data. This is because the support vectors tend to be instances on the edge of the class boundaries and also instances that are classified incorrectly in the training data. 

You can also look at joint plots of the data and see how relationships have changed. (**Hint hint for the min-lab assignment**--this would be a nice analysis of the support vectors.)

That's mostly it for using these things! They are really nice analysis tools and provide human interpretable summaries of the data. 
___

# Gradient Based Alternatives
So now let's go and find out how we can use these when our data size gets bigger. Like a lot bigger. We will use a kaggle dataset that attempts to classify plankton. We will use some example code to get us started from the tutorial here:
http://www.kaggle.com/c/datasciencebowl/details/tutorial 

You can download from Kaggle (login required): https://www.kaggle.com/c/datasciencebowl/data

UPDATE: This problem was also solved using deep learning! Check out the blog here:
http://benanne.github.io/2015/03/17/plankton.html 

In [ ]:
Image(url='https://storage.googleapis.com/kaggle-competitions/kaggle/3978/media/Plankton-Diagram3-lg.png')    

In [ ]:
# load another dataset (large) and train using various methods of gradient (and mini-batch)
from __future__ import print_function

import glob
import os


# change this to point to the dataset on your machine/cluster!!
# For my students: please email me for a link to the data if you cannot get it from Kaggle
directory_of_dataset = "D:/SMU/Larson/2019 Data Mining 7331 Updates/Class Github/kaggle_plank/"

# get the classnames from the training data directory structure
directory_names = list(set(glob.glob(os.path.join(directory_of_dataset,"train", "*"))
 ).difference(set(glob.glob(os.path.join(directory_of_dataset,"train","*.*")))))

print('number of classes:', len(directory_names))

In [ ]:
# using this code requires the use of skimage to process the images (you will need to install via pip)
import pandas as pd
import numpy as np

# Rescale the images and create the combined metrics and training labels
from skimage.transform import resize
from skimage.io import imread

#get the total training images
numberofImages = 0
for folder in directory_names:
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
             # Only read in the images
            if fileName[-4:] != ".jpg":
              continue
            numberofImages += 1

# We'll rescale the images to be 40x40
maxPixel = 25
imageSize = maxPixel * maxPixel
num_rows = numberofImages # one row for each image in the training dataset
num_features = imageSize # for our ratio

# X is the feature vector with one row of features per image
# consisting of the pixel values and our metric
X = np.zeros((num_rows, num_features), dtype=float)
# y is the numeric class label 
y = np.zeros((num_rows))

files = []
# Generate training data
i = 0    
label = 0
# List of string of class names
namesClasses = list()

print("Reading images")
# Navigate through the list of directories
for folder in directory_names:
    # Append the string class name for each class
    currentClass = folder.split(os.pathsep)[-1]
    namesClasses.append(currentClass)
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
            # Only read in the images
            if fileName[-4:] != ".jpg":
              continue
            
            # Read in the images and create the features
            nameFileImage = "{0}{1}{2}".format(fileNameDir[0], os.sep, fileName)            
            image = imread(nameFileImage, as_gray=True)
            files.append(nameFileImage)
            #axisratio = getMinorMajorRatio(image)
            image = resize(image, (maxPixel, maxPixel))
            
            # Store the rescaled image pixels and the axis ratio
            X[i, 0:imageSize] = np.reshape(image, (1, imageSize))
            #X[i, imageSize] = axisratio
            
            # Store the classlabel
            y[i] = label
            i += 1
            # report progress for each 5% done  
            report = [int((j+1)*num_rows/20.) for j in range(20)]
            if i in report: print(np.ceil(i *100.0 / num_rows), "% done")
    label += 1

In [ ]:
# here is where the online tutorial code stops and my code starts
print(X.shape)
print(y.shape)

In [ ]:
# now divide the data into test and train using scikit learn built-ins
from sklearn.model_selection import StratifiedShuffleSplit 

cv = StratifiedShuffleSplit( n_splits=1,test_size=0.5)

In [ ]:
%%time
# use some compact notation for creating a linear SVM classifier with stichastic descent
from sklearn.linear_model import SGDClassifier

regularize_const = 0.1
iterations = 5
svm_sgd = SGDClassifier(alpha=regularize_const,
        fit_intercept=True, l1_ratio=0.0, learning_rate='optimal',
        loss='hinge', n_iter_no_change=iterations, n_jobs=-1, penalty='l2')

scl = StandardScaler()
for train_idx, test_idx in cv.split(X,y):
    svm_sgd.fit(scl.fit_transform(X[train_idx]),y[train_idx])
    yhat = svm_sgd.predict(scl.transform(X[test_idx]))
    
    conf = mt.confusion_matrix(y[test_idx],yhat)
    acc = mt.accuracy_score(y[test_idx],yhat)

print('SVM:', acc)

In [ ]:
%%time
# use some compact notation for creating a logistic regression classifier with stochastic descent
log_sgd = SGDClassifier(alpha=regularize_const,
        fit_intercept=True, l1_ratio=0.0, learning_rate='optimal',
        loss='log', n_iter_no_change=iterations, n_jobs=-1, penalty='l2')

scl = StandardScaler()
for train_idx, test_idx in cv.split(X,y):
    log_sgd.fit(scl.fit_transform(X[train_idx]),y[train_idx])
    yhat = log_sgd.predict(scl.transform(X[test_idx]))
    
    conf = mt.confusion_matrix(y[test_idx],yhat)
    acc = mt.accuracy_score(y[test_idx],yhat)

print('Logistic Regression:', acc)

In [ ]:
%%time
# now lets use some of what we know from this class to reduce the dimensionality of the set
from sklearn.decomposition import PCA
n_components = 50

pca = PCA(n_components=n_components, svd_solver='randomized')

iterations = 150
log_sgd = SGDClassifier(
        fit_intercept=True, l1_ratio=0.0, learning_rate='optimal',
        loss='log', n_iter=iterations, n_jobs=-1, penalty='l2')

# you could also set this up in a pipeline
for train_idx, test_idx in cv.split(X,y):
    log_sgd.fit(pca.fit_transform(X[train_idx]),y[train_idx])
    yhat = log_sgd.predict(pca.transform(X[test_idx]))
    
    conf = mt.confusion_matrix(y[test_idx],yhat)
    acc = mt.accuracy_score(y[test_idx],yhat)

print('Logistic Regression:', acc)

In [ ]:
f = plt.figure(figsize=(8,8))
plt.imshow(conf)

The competition does not use "accuracy" as the evaluation of the best model; they use the log loss:
$$logloss=-\frac{1}{N}\sum_{i=1}^m\sum_{j=1}^C {\bf 1}_{ij}\ln(p_{ij})$$

Where there are $m$ instances (images) in the dataset, and $C$ is the number of classes. The equation ${\bf 1}_{ij}$ is an indicator function that ensures we only add log probabilities when the class is correct. That is, it is zero if the predicted class for the $i^{th}$ instance is not equal to $j$ and it is one when the class of the $i^{th}$ instance == $j$. To prevent extremities in the log function they also replace probabilities, $p$, with $p=\max(\min(p,1-10^{-15}),10^{-15})$

Would this be easy to code in python? `scikit-learn` has an implementation for log loss, but it is not exactly what the competition uses and is only defined for binary classes.

In [ ]:
# from the tutorial: http://www.kaggle.com/c/datasciencebowl/details/tutorial 
def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    https://www.kaggle.com/wiki/MultiClassLogLoss

    Parameters
    ----------
    y_true : array, shape = [n_samples]
            true class, integers in [0, n_classes - 1)
    y_pred : array, shape = [n_samples, n_classes]

    Returns
    -------
    loss : float
    """
    predictions = np.clip(y_pred, eps, 1 - eps)

    # normalize row sums to 1
    predictions /= predictions.sum(axis=1)[:, np.newaxis]

    actual = np.zeros(y_pred.shape)
    n_samples = actual.shape[0]
    actual[np.arange(n_samples), y_true.astype(int)] = 1
    vectsum = np.sum(actual * np.log(predictions))
    loss = -1.0 / n_samples * vectsum
    return loss

___
## How do you think you might increase the accuracy of the classifier(s)?

Normalize your data. Try to reduce the dimensions as much as you can.
#https://www.analyticsvidhya.com/blog/2015/12/improve-machine-learning-results/
-Add more data
-Treat missing and Outlier values
-Feature Engineering : This step helps to extract more information from existing data. New information is extracted in terms of new features. These features may have a higher ability to explain the variance in the training data. Thus, giving improved model accuracy.Feature engineering is highly influenced by hypotheses generation. Good hypothesis result in good features. That’s why, I always suggest to invest quality time in hypothesis generation. Feature engineering process can be divided into two steps
-Feature Selection : 
- Multiple algorithms
-Hitting at the right machine learning algorithm is the ideal approach to achieve higher accuracy: Regression, classification and clustering
- Algorithm Tuning : We know that machine learning algorithms are driven by parameters. These parameters majorly influence the outcome of learning process.The objective of parameter tuning is to find the optimum value for each parameter to improve the accuracy of the model. To tune these parameters, you must have a good understanding of these meaning and their individual impact on model. You can repeat this process with a number of well performing models.For example: In random forest, we have various parameters like max_features, number_trees, random_state, oob_score and others. Intuitive optimization of these parameter values will result in better and more accurate models.
- Ensemble methods: This is the most common approach found majorly in winning solutions of Data science competitions. This technique simply combines the result of multiple weak models and produce better results. This can be achieved through many ways:Bagging (Bootstrap Aggregating), Boosting.
- Cross Validation

- Search through parameters for models?
- Try different classifiers?
- Add more features (through better image processing)?

___
## How do you think we can make the algorithms more efficient for training/testing?
- What about mini-batch training? 
- Sampling?
- Map/Reduce (what are advantages/disadvantages)?
- Buy a ton of memory on AWS virtual machines?

**Note:** For mini-batch calculations (they are not really needed here because the dataset fits in memory) they can be accessed for a number of different classifiers (including SGDClassifier) by managing the sub-samples we send it, $X_{sub}$, and calling the function `partial_fit`.